In [17]:
""" Inspired by example from
https://github.com/Vict0rSch/deep_learning/tree/master/keras/recurrent
Uses the TensorFlow backend
The basic idea is to detect anomalies in a time-series.
"""
import matplotlib as mpl
mpl.use('TkAgg')
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import time
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.recurrent import LSTM
from keras.models import Sequential
from numpy import arange, sin, pi, random

np.random.seed(1234)

# Global hyper-parameters
sequence_length = 100
random_data_dup = 10  # each sample randomly duplicated between 0 and 9 times, see dropin function
epochs = 1
batch_size = 50






In [16]:
X_train, y_train, X_test, y_test = get_split_prep_data(0, 700, 500, 1000)
X_train.shape, y_train.shape, X_test.shape, y_test.shape

wave1 1000
wave2 1000
wave3 50
Length of Data 1000
Creating train data...
Mean of train data :  0.0020295689605
Train data shape  :  (600, 100)
X shape: (600, 99)
y shape: (600,)
Creating test data...
Mean of test data :  0.0109595003004
Test data shape  :  (400, 100)
Shape X_train (3018, 99)
Shape X_test (400, 99)


/Users/june/Documents/tf/lib/python3.6/site-packages/ipykernel_launcher.py:13: DeprecationWarning: This function is deprecated. Please call randint(0, 10 + 1) instead
  del sys.path[0]


((3018, 99, 1), (3018,), (400, 99, 1), (400,))

In [30]:
v1 = pd.read_csv('test_v1.csv')

In [52]:
v1.drop('Time', axis=1, inplace=True)
v1.drop('Date', axis=1, inplace=True)

ValueError: labels ['Date'] not contained in axis

In [53]:
v1.sort_index(ascending=False, inplace=True)

In [54]:
v1 = v1.reset_index()

In [55]:
v1.drop('index',axis=1, inplace=True)

In [58]:
v1.fillna(-1, inplace=True)

In [62]:
active_power = v1['Global_active_power']

In [75]:
wave = gen_wave()
wave.shape, active_power.values.shape

wave1 1000
wave2 1000
wave3 50


((1000,), (786422,))

In [79]:
X_train, y_train, X_test, y_test = get_split_prep_data(active_power.values, 0, 2000, 2000, 2500)
X_train.shape, y_train.shape, X_test.shape, y_test.shape


Length of Data 786422
Creating train data...
Mean of train data :  2.57262071621
Train data shape  :  (1900, 100)
X shape: (1900, 99)
y shape: (1900,)
Creating test data...
Mean of test data :  1.48893302501
Test data shape  :  (400, 100)
Shape X_train (9512, 99)
Shape X_test (400, 99)


/Users/june/Documents/tf/lib/python3.6/site-packages/ipykernel_launcher.py:13: DeprecationWarning: This function is deprecated. Please call randint(0, 10 + 1) instead
  del sys.path[0]


((9512, 99, 1), (9512,), (400, 99, 1), (400,))

In [2]:
def dropin(X, y):
    """ The name suggests the inverse of dropout, i.e. adding more samples. See Data Augmentation section at
    http://simaaron.github.io/Estimating-rainfall-from-weather-radar-readings-using-recurrent-neural-networks/
    :param X: Each row is a training sequence
    :param y: Tne target we train and will later predict
    :return: new augmented X, y
    """
    print("X shape:", X.shape)
    print("y shape:", y.shape)
    X_hat = []
    y_hat = []
    for i in range(0, len(X)):
        for j in range(0, np.random.random_integers(0, random_data_dup)):
            X_hat.append(X[i, :])
            y_hat.append(y[i])
    return np.asarray(X_hat), np.asarray(y_hat)



In [4]:

def z_norm(result):
    result_mean = result.mean()
    result_std = result.std()
    result -= result_mean
    result /= result_std
    return result, result_mean



In [77]:

def get_split_prep_data(data, train_start, train_end, test_start, test_end):
    print("Length of Data", len(data))

    # train data
    print ("Creating train data...")

    result = []
    for index in range(train_start, train_end - sequence_length):
        result.append(data[index: index + sequence_length])
    result = np.array(result)  # shape (samples, sequence_length)
    result, result_mean = z_norm(result)

    print ("Mean of train data : ", result_mean)
    print ("Train data shape  : ", result.shape)

    train = result[train_start:train_end, :]
    np.random.shuffle(train)  # shuffles in-place
    X_train = train[:, :-1]
    y_train = train[:, -1]
    X_train, y_train = dropin(X_train, y_train)

    # test data
    print ("Creating test data...")

    result = []
    for index in range(test_start, test_end - sequence_length):
        result.append(data[index: index + sequence_length])
    result = np.array(result)  # shape (samples, sequence_length)
    result, result_mean = z_norm(result)

    print ("Mean of test data : ", result_mean)
    print ("Test data shape  : ", result.shape)

    X_test = result[:, :-1]
    y_test = result[:, -1]

    print("Shape X_train", np.shape(X_train))
    print("Shape X_test", np.shape(X_test))

    X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
    X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

    return X_train, y_train, X_test, y_test



In [6]:
def build_model():
    model = Sequential()
    layers = {'input': 1, 'hidden1': 64, 'hidden2': 256, 'hidden3': 100, 'output': 1}

    model.add(LSTM(
            input_shape=(sequence_length - 1, layers['input']),
            output_dim=layers['hidden1'],
            return_sequences=True))
    model.add(Dropout(0.2))

    model.add(LSTM(
            layers['hidden2'],
            return_sequences=True))
    model.add(Dropout(0.2))

    model.add(LSTM(
            layers['hidden3'],
            return_sequences=False))
    model.add(Dropout(0.2))

    model.add(Dense(
            units=layers['output']))
    model.add(Activation("linear"))

    start = time.time()
    model.compile(loss="mse", optimizer="rmsprop")
    print ("Compilation Time : ", time.time() - start)
    return model



In [7]:

def run_network(model=None, data=None):

    return model, y_test, predicted


In [ ]:
model = None
data = active_power.values
global_start_time = time.time()
X_train, y_train, X_test, y_test = get_split_prep_data(active_power.values, 0, 2000, 2000, 2500)

print ('\nData Loaded. Compiling...\n')

if model is None:
    model = build_model()

try:
    print("Training...")
    model.fit(
            X_train, y_train,
            batch_size=batch_size, epochs=epochs, validation_split=0.05)
    print("Predicting...")
    predicted = model.predict(X_test)
    print("Reshaping predicted")
    predicted = np.reshape(predicted, (predicted.size,))
except KeyboardInterrupt:
    print("prediction exception")
    print ('Training duration (s) : ', time.time() - global_start_time)
    
try:
    plt.figure(1)
    plt.subplot(311)
    plt.title("Actual Test Signal w/Anomalies")
    plt.plot(y_test[:len(y_test)], 'b')
    plt.subplot(312)
    plt.title("Predicted Signal")
    plt.plot(predicted[:len(y_test)], 'g')
    plt.subplot(313)
    plt.title("Squared Error")
    mse = ((y_test - predicted) ** 2)
    plt.plot(mse, 'r')
    plt.show()
except Exception as e:
    print("plotting exception")
    print (str(e))
print ('Training duration (s) : ', time.time() - global_start_time)


Length of Data 786422
Creating train data...
Mean of train data :  2.57262071621
Train data shape  :  (1900, 100)
X shape: (1900, 99)
y shape: (1900,)
Creating test data...
Mean of test data :  1.48893302501
Test data shape  :  (400, 100)
Shape X_train (9482, 99)
Shape X_test (400, 99)

Data Loaded. Compiling...



/Users/june/Documents/tf/lib/python3.6/site-packages/ipykernel_launcher.py:13: DeprecationWarning: This function is deprecated. Please call randint(0, 10 + 1) instead
  del sys.path[0]
/Users/june/Documents/tf/lib/python3.6/site-packages/ipykernel_launcher.py:8: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(input_shape=(99, 1), return_sequences=True, units=64)`
  


Compilation Time :  0.050057172775268555
Training...
Train on 9007 samples, validate on 475 samples
Epoch 1/1
9007/9007 [==============================] - 425s - loss: 0.4055 - val_loss: 0.3269
Predicting...
Reshaping predicted


(400, 99, 1)